<a href="https://colab.research.google.com/github/chris051091/Seminario-3/blob/master/Analysis/Eventos_Estaciones/Analisis_Eventos_Palanquero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import io

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
PATH_DRIVE = 'drive/MyDrive/Seminario-3-master'
PATH_DADOS_EVENTOS = '/dados/'
FILE_DATOS_ESTACION = 'eventos_palanquero.csv'

PATH_DADOS_PREPROCESS = '/preprocess/dados/palanquero/tratados/'
FILE_TEMP = 'palanquero_tratados_temp.csv'
FILE_HUM = 'palanquero_tratados_hum.csv'
FILE_PRES = 'palanquero_tratados_pres.csv'
FILE_PREC = 'palanquero_tratados_prec.csv'
FILE_VIENTO = 'palanquero_tratados_viento.csv'

# **Cargando eventos**

In [64]:
dfEventos = pd.read_csv(PATH_DRIVE + PATH_DADOS_EVENTOS + FILE_DATOS_ESTACION, 
                 sep = ';'
                )
dfEventos = dfEventos.sort_values(by='date_min_t').reset_index(drop = True)
dfEventos['FECHA'] = pd.to_datetime(dfEventos['date_min_t'], format='%Y %m %d %H %M').dt.tz_localize('America/Bogota')
dfEventos['FECHA_MIN'] = dfEventos['FECHA'].dt.round('10min')  
dfEventos['FECHA'] = dfEventos['FECHA'].dt.round('H')
dfEventos

,date_min_t,lon_t,lat_t,min_t,hora,lon_t_palanquero,lat_t_palanquero,distancia,FECHA,FECHA_MIN
0,2011 06 06 22 15,-74.693,5.484,198.5,22,-74.692857,5.483846,0.00021,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00
1,2011 11 03 02 15,-74.693,5.484,191.9,2,-74.692857,5.483846,0.00021,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00
2,2011 12 11 02 45,-74.693,5.484,199.3,2,-74.692857,5.483846,0.00021,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00
3,2012 03 16 22 45,-74.693,5.484,193.6,22,-74.692857,5.483846,0.00021,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00
4,2012 09 11 04 45,-74.693,5.484,217.9,4,-74.692857,5.483846,0.00021,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00
5,2013 07 05 01 15,-74.693,5.484,226.0,1,-74.692857,5.483846,0.00021,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00
6,2013 11 11 00 15,-74.693,5.484,212.0,0,-74.692857,5.483846,0.00021,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00
7,2013 11 11 00 45,-74.693,5.484,208.5,0,-74.692857,5.483846,0.00021,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00
8,2014 10 19 00 45,-74.693,5.484,196.3,0,-74.692857,5.483846,0.00021,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00
9,2014 12 07 23 45,-74.693,5.484,219.2,23,-74.692857,5.483846,0.00021,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00


In [67]:
FILES = [FILE_PRES, FILE_HUM, FILE_TEMP]
df = dfEventos.copy()

for FILE in FILES :
  
  dfVar = pd.read_csv(PATH_DRIVE + PATH_DADOS_PREPROCESS + FILE, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                )
  
  df = pd.merge(df, dfVar, on=['FECHA'], how = 'left')
  df['FECHA'] = pd.to_datetime(df["FECHA"])

In [68]:
VARS = ['TEMP', 'PRESION', 'HUM']

for VAR in VARS:
  df[f'{VAR}_MEAN'] = df[VAR].mean()

df

,date_min_t,lon_t,lat_t,min_t,hora,lon_t_palanquero,lat_t_palanquero,distancia,FECHA,FECHA_MIN,PRESION,HUM,TEMP,TEMP_MEAN,PRESION_MEAN,HUM_MEAN
0,2011 06 06 22 15,-74.693,5.484,198.5,22,-74.692857,5.483846,0.00021,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333
1,2011 11 03 02 15,-74.693,5.484,191.9,2,-74.692857,5.483846,0.00021,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00,988.4,NaN,25.6,24.64,989.86,87.333333
2,2011 12 11 02 45,-74.693,5.484,199.3,2,-74.692857,5.483846,0.00021,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333
3,2012 03 16 22 45,-74.693,5.484,193.6,22,-74.692857,5.483846,0.00021,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333
4,2012 09 11 04 45,-74.693,5.484,217.9,4,-74.692857,5.483846,0.00021,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00,990.2,80.0,24.6,24.64,989.86,87.333333
5,2013 07 05 01 15,-74.693,5.484,226.0,1,-74.692857,5.483846,0.00021,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00,991.3,98.0,24.9,24.64,989.86,87.333333
6,2013 11 11 00 15,-74.693,5.484,212.0,0,-74.692857,5.483846,0.00021,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00,990.2,NaN,25.0,24.64,989.86,87.333333
7,2013 11 11 00 45,-74.693,5.484,208.5,0,-74.692857,5.483846,0.00021,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00,990.2,NaN,23.4,24.64,989.86,87.333333
8,2014 10 19 00 45,-74.693,5.484,196.3,0,-74.692857,5.483846,0.00021,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00,993.7,100.0,22.2,24.64,989.86,87.333333
9,2014 12 07 23 45,-74.693,5.484,219.2,23,-74.692857,5.483846,0.00021,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00,989.6,98.0,24.9,24.64,989.86,87.333333


In [69]:
FILES = [FILE_PREC, FILE_VIENTO]

df10Min = df.copy()
for FILE in FILES :
  
  dfVar = (pd.read_csv(PATH_DRIVE + PATH_DADOS_PREPROCESS + FILE, 
                 sep = ';', 
                 parse_dates = ["FECHA"],
                 date_parser = lambda col: pd.to_datetime(col)
                ).rename(columns = {'FECHA' : 'FECHA_MIN'})
          )
  
  df10Min = pd.merge(df10Min, dfVar, on=['FECHA_MIN'], how = 'left')
  df10Min['FECHA_MIN'] = pd.to_datetime(df["FECHA_MIN"])

In [70]:
df10Min

,date_min_t,lon_t,lat_t,min_t,hora,lon_t_palanquero,lat_t_palanquero,distancia,FECHA,FECHA_MIN,PRESION,HUM,TEMP,TEMP_MEAN,PRESION_MEAN,HUM_MEAN,PREC,DIRECVIENTO,VELVIENTO
0,2011 06 06 22 15,-74.693,5.484,198.5,22,-74.692857,5.483846,0.00021,2011-06-06 22:00:00-05:00,2011-06-06 22:20:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333,NaN,NaN,NaN
1,2011 11 03 02 15,-74.693,5.484,191.9,2,-74.692857,5.483846,0.00021,2011-11-03 02:00:00-05:00,2011-11-03 02:20:00-05:00,988.4,NaN,25.6,24.64,989.86,87.333333,0.0,NaN,NaN
2,2011 12 11 02 45,-74.693,5.484,199.3,2,-74.692857,5.483846,0.00021,2011-12-11 03:00:00-05:00,2011-12-11 02:40:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333,NaN,NaN,NaN
3,2012 03 16 22 45,-74.693,5.484,193.6,22,-74.692857,5.483846,0.00021,2012-03-16 23:00:00-05:00,2012-03-16 22:40:00-05:00,NaN,NaN,NaN,24.64,989.86,87.333333,NaN,NaN,NaN
4,2012 09 11 04 45,-74.693,5.484,217.9,4,-74.692857,5.483846,0.00021,2012-09-11 05:00:00-05:00,2012-09-11 04:40:00-05:00,990.2,80.0,24.6,24.64,989.86,87.333333,0.0,NaN,NaN
5,2013 07 05 01 15,-74.693,5.484,226.0,1,-74.692857,5.483846,0.00021,2013-07-05 01:00:00-05:00,2013-07-05 01:20:00-05:00,991.3,98.0,24.9,24.64,989.86,87.333333,0.0,56.0,0.2
6,2013 11 11 00 15,-74.693,5.484,212.0,0,-74.692857,5.483846,0.00021,2013-11-11 00:00:00-05:00,2013-11-11 00:20:00-05:00,990.2,NaN,25.0,24.64,989.86,87.333333,16.7,251.0,0.5
7,2013 11 11 00 45,-74.693,5.484,208.5,0,-74.692857,5.483846,0.00021,2013-11-11 01:00:00-05:00,2013-11-11 00:40:00-05:00,990.2,NaN,23.4,24.64,989.86,87.333333,6.9,104.0,0.3
8,2014 10 19 00 45,-74.693,5.484,196.3,0,-74.692857,5.483846,0.00021,2014-10-19 01:00:00-05:00,2014-10-19 00:40:00-05:00,993.7,100.0,22.2,24.64,989.86,87.333333,1.0,98.0,0.2
9,2014 12 07 23 45,-74.693,5.484,219.2,23,-74.692857,5.483846,0.00021,2014-12-08 00:00:00-05:00,2014-12-07 23:40:00-05:00,989.6,98.0,24.9,24.64,989.86,87.333333,0.0,30.0,0.7
